# Multi_agent_preference_matrix_generation
In this notebook the enviornment lattice for each agent is generated from the simulation matrices done in the enviornment lattices notebooks.

## Initilization
Libraries used in this notebook :<br/>
1.Topsis for MCDA [https://scikit-criteria.readthedocs.io/en/latest/tutorial/quickstart.html]<br/>
2.Ipywidgets for interactivity [https://minrk-ipywidgets.readthedocs.io/en/latest/index.html]<br/>

In [1]:
import os
import sys
import pickle
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import collections 
from itertools import cycle 
import sys
from math import factorial as fac
from skcriteria import Data, MIN, MAX
from skcriteria.madm import closeness, simple ,moora , electre
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual , Layout
import ipywidgets as widgets
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

In [3]:
flattened_lattice = envelope_lattice.flatten()

### Define Widget styles

In [4]:
layout = widgets.Layout(width='auto', height='auto') #set width and height
style = {'description_width': 'initial'} # Display full text

## Define the Agents in the system 

 Define the number of Agents and their name in the project by editing the widgets in the notebook

In [5]:
Agent_number_widget = widgets.BoundedIntText(
    value=1,
    min=0,
    max=10,
    step=1,
    description='Number of Agents:',
    style=style,
    disabled=False
)

In [6]:
Agent_number_widget

BoundedIntText(value=1, description='Number of Agents:', max=10, style=DescriptionStyle(description_width='ini…

In [36]:
Agent_number = Agent_number_widget.value
Agent_numbering = []
for s in range(Agent_number): 
    a= "Stakeholder"+ str(s+1)
    Agent_numbering.append(a)

### Input the name for the Agents in the tab widget

In [37]:
tab_contents = Agent_numbering
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, str(i))
tab

In [182]:
Agents_list = []
for i in range(len(children)):
    Agents_list.append(tab.children[i].value)

In [183]:
print (Agents_list)

['Privately Owned Housing', 'Social Sector Rental Housing', 'Free Sector Rental Housing', 'Restaurants and Cafe', 'Retail Stores', 'Offices', 'Parking']


In [184]:
Agents_dropdown = widgets.Dropdown(
    options= Agents_list,
    value= str(Agents_list[0]) ,
    description='Number:',
    disabled=False,
)

## Select the Agent for recording design preference

In [185]:
Agents_dropdown

Dropdown(description='Number:', options=('Privately Owned Housing', 'Social Sector Rental Housing', 'Free Sect…

## Selection of appropriate Design criteria 

The appropriate design criteria should be selected for each stakeholder to create a design preference which is a collection of various simulation lattices calulated for the site

In [186]:
Design_criteria_selection_options=['Closeness to facade N', 'Closeness to facade S', 'Closeness to facade W','Closeness to facade E','Closeness to Roof','Closeness to Ground','Quiteness', 'Visibility to IJ', 'Visibility from street', 'Solar access']

### Selection of appropriate Weights for the Design Criteria

In [187]:
def criteria_slider(name):
    criteria_slider=widgets.IntSlider(   
    value=0,
    min=0,
    max=10,
    step=1,
    description=str(name),
    style=style,
    layout=Layout(width='500px'),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
    return criteria_slider

In [188]:
slider_lists=[]
for a in (Design_criteria_selection_options):
    slider_lists.append(criteria_slider(a))

In [189]:
option_list= list(range(len(Design_criteria_selection_options)))


In [190]:
ui = widgets.GridBox(slider_lists,layout=widgets.Layout(grid_template_rows="auto"))
display(ui)

GridBox(children=(IntSlider(value=0, continuous_update=False, description='Closeness to facade N', layout=Layo…

In [204]:
Topsis_weights =[]

for slider in  ui.children:
    Topsis_weights.append((slider.value)/10)

print(Topsis_weights)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.9, 0.7, 0.0, 0.9]


In [205]:
def Radio_widget(text):
    Radio_widget = widgets.RadioButtons(
    options=['Cost', 'Benefit'],
    value='Benefit',
    description=str(text),
    disabled=False,
    style=style,
    layout=Layout(width='500px')
)
    return Radio_widget

In [206]:
Radio_lists=[]
for a in (Design_criteria_selection_options):
    Radio_lists.append(Radio_widget(a))

In [207]:
ui_radios = widgets.GridBox(Radio_lists,layout=widgets.Layout(grid_template_rows="auto"))
display(ui_radios)

GridBox(children=(RadioButtons(description='Closeness to facade N', index=1, layout=Layout(width='500px'), opt…

In [208]:
criteria =[-1, -1, -1, -1, -1, -1, 1, 1, 1, 1]
"""
for radio in ui_radios.children:
    if radio.value == "Cost":
        criteria.append(MIN)
    else:
        criteria.append(MAX)

print(criteria)
"""

'\nfor radio in ui_radios.children:\n    if radio.value == "Cost":\n        criteria.append(MIN)\n    else:\n        criteria.append(MAX)\n\nprint(criteria)\n'

### Importing all the necessary Lattices into the Notebook

It is necessary to flatten the matrixes for combining and MCDA

In [209]:
North_facade_lattice = (pickle.load( open( "North_facade_lattice.p", "rb" ) )).flatten() 
South_facade_lattice = (pickle.load( open( "South_facade_lattice.p", "rb" ) ) ).flatten()
East_facade_lattice = (pickle.load( open( "East_facade_lattice.p", "rb" ) )) .flatten()
West_facade_lattice = (pickle.load( open( "West_facade_lattice.p", "rb" ) )) .flatten()
Roof_lattice = (pickle.load( open( "Distance_from_roof.p", "rb" ) )) .flatten()
Ground_lattice = (pickle.load( open( "Distance_from_ground.p", "rb" ) )) .flatten()
Quiteness_lattice = (pickle.load( open( "Quiteness_lattice.p", "rb" ) )) .flatten()
Visibility_to_IJ_lattice = (pickle.load( open( "Visibility_IJ_lattice.p", "rb" ) )) .flatten()
Visibility_to_Road_lattice = (pickle.load( open( "Visibility_Road_lattice.p", "rb" ) )) .flatten()
Sun_access_lattice = (pickle.load( open( "Solar_access_lattice.p", "rb" ) )) .flatten()


In [210]:
array_all_lattices =[North_facade_lattice,South_facade_lattice,East_facade_lattice,West_facade_lattice,Roof_lattice,Ground_lattice,Quiteness_lattice,Visibility_to_IJ_lattice,Visibility_to_Road_lattice,Sun_access_lattice]
np_array_all_lattices =np.array(array_all_lattices).T

In [211]:
data = Data(np_array_all_lattices, criteria,          
            weights=Topsis_weights,
            cnames=Design_criteria_selection_options)

In [212]:
topsis_method = closeness.TOPSIS()
topsis_decision = topsis_method.decide(data)
agent_preference = topsis_decision.e_.closeness*10

In [213]:
Agent_preference_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(agent_preference))])

In [214]:
Agent_preference_array = np.array(Agent_preference_lattice_padded)

In [215]:
Agent_preference_np = Agent_preference_array.reshape(envelope_lattice.shape)

In [216]:
Agent_preference_lattice =tg.to_lattice(Agent_preference_np, Agent_preference_np.shape)

### Vizualise the decision matrix for the user 

In [217]:
#Design Details
Self_development_plots_path = os.path.relpath('Site_self_development_plots.obj')
Self_development_backyards_path = os.path.relpath('Site_self_development_backyards.obj')
Site_buildings_path = os.path.relpath('Site_buildings.obj')
Site_green_areas_path = os.path.relpath('Site_green_areas.obj')
Site_roads_path = os.path.relpath('Site_base_roads.obj')
Site_context_shading_path= os.path.relpath('Site_surrounding_buildings_for_shading.obj') 


# Site details
Site_base_path = os.path.relpath('Site_base_block.obj')
Site_surrounding_buildings_path = os.path.relpath('Site_surrounding_buildings.obj')
Site_water_bodies_path = os.path.relpath('Site_water_bodies.obj')
Site_roads_path = os.path.relpath('Site_roads.obj')
Site_other_buildings_path = os.path.relpath('Site_other_buildings.obj')

# load the mesh from file
# Design elements
Self_development_plots_mesh = tm.load(Self_development_plots_path)
Self_development_backyards_mesh = tm.load(Self_development_backyards_path)
Site_building_mesh = tm.load(Site_buildings_path)
Site_green_areas_mesh = tm.load(Site_green_areas_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_context_shading_mesh = tm.load(Site_context_shading_path)

#Site elements
Site_base_mesh = tm.load(Site_base_path)
Site_surrounding_buildings_mesh = tm.load(Site_surrounding_buildings_path)
Site_water_bodies_mesh = tm.load(Site_water_bodies_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_other_buildings_mesh = tm.load(Site_other_buildings_path)


# Check if the mesh is watertight
#print(envelope_mesh.is_watertight)
#print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


In [218]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

In [219]:
Agent_name = str(Agents_dropdown.value)

In [220]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays[(str(Agent_name)+"_preference")] = Agent_preference_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume
opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="inferno", opacity=opacity, shade=False)
# plotting
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot=((str(Agent_name))+".png"))

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

### Save the Agent preference lattice as Pickle
The lattice for the selected Stakeholder will be saved as pickle object Continue this process for all the users and make the lattices for all of them 

In [221]:
Agent_mtrx = pickle.dump( Agent_preference_lattice, open( (str(Agent_name)+".p"), "wb" ) )